Challenge: we need to build a logic to estimate all products that are duplicated, ie, they have different product_ids but they share the same content.

### we don’t want the customer to see duplicated products.

The unique identifier for each product on Cornershop database is the **PRODUCT_ID** attribute.

# Script

### Libraries and File upload

In [2]:
import pandas as pd
from pathlib import Path

In [3]:
home = str(Path.home())

In [4]:
path_read = home + '/Downloads/Base de Dados Python.csv'
df_database = pd.read_csv(path_read, dtype={'product_id': 'string',
                                            'product_name': 'string',
                                            'buy_unit': 'string',
                                            'package': 'string',
                                            'brand': 'string',
                                            'parent_category_id':'string',
                                            'category_id':'string',
                                            'parent_category_name': 'string',
                                            'category_name': 'string',
                                            'orders;;':'string'})

In [5]:
pd.options.mode.chained_assignment = None  # default='warn'

### Functions

In [6]:
def splitcom(value):
    value1 = value.split(',')
    return value1

def text_char(x):
    text_char =  unidecode.unidecode(str(x))
    return text_char

def order_clean(x):
    order_clean = str(x).replace(';','')
    return order_clean

def no_elements(list):
    count = 0
    for element in list:
        count += 1
    return count

# Data manipulation

In [7]:
df_database.head(3)

,product_id,product_name,buy_unit,package,brand,parent_category_id,category_id,parent_category_name,category_name,orders;;
0,1,At√∫n en aceite,UN,3 u x 160 g c/u,Van Camp's,521,83,Productos en conserva,Carnes y mariscos en conserva,262;;
1,2,At√∫n en aceite,UN,1730 g,Van Camp's,521,83,Productos en conserva,Carnes y mariscos en conserva,387;;
2,3,At√∫n en aceite de girasol,UN,4 u x 160 g c/u,Van Camp's,521,83,Productos en conserva,Carnes y mariscos en conserva,600;;


In [8]:
df_len = df_database
df_len['len'] = df_len['product_id'].apply(lambda x: len(x))
df_len.rename(columns = {'orders;;': 'orders'}, inplace = True)

# Dataset ok
df_len_ok = df_len[(df_len['len']) <= 6 ]
df_len_ok = df_len_ok.drop(['len'],axis=1)

# Dataset nok
df_len_bad = df_len[(df_len['len']) > 7 ]
df_len_bad['split_list'] = df_len_bad['product_id'].apply(lambda x: splitcom(x))
df_len_bad['no_elements'] = df_len_bad['split_list'].apply(lambda x: no_elements(x))

# Dataset nok, 10 elements
df_len_bad_10 = df_len_bad[(df_len_bad['no_elements'] == 10)]
df_len_bad_10 = pd.DataFrame(df_len_bad_10['product_id'])
df_len_bad_10[['product_id_new', 'product_name','buy_unit', 'package', 'brand','parent_category_id', 'category_id','parent_category_name', 'category_name','orders']] = df_len_bad_10['product_id'].str.split(',', expand=True)
df_len_bad_10 = df_len_bad_10.drop(['product_id'],axis=1)
df_len_bad_10.rename(columns = {'product_id_new': 'product_id'}, inplace = True)
df_len_bad_10['product_name'] = df_len_bad_10['product_name'].apply(lambda x: x.replace('"',''))
df_len_bad_10['package'] = df_len_bad_10['package'].apply(lambda x: x.replace('"',''))
df_len_bad_10 = df_len_bad_10[pd.to_numeric(df_len_bad_10['product_id'], errors='coerce').notnull()]
df_len_bad_10 = df_len_bad_10[(df_len_bad_10['buy_unit'] == 'UN') | (df_len_bad_10['buy_unit'] == 'KG')]

## Appending datasets

In [9]:
df_cleaned = df_len_ok.append(df_len_bad_10)

# Additional cleaning
df_cleaned['product_id'] = df_cleaned['product_id'].astype(int)
df_cleaned['parent_category_id'] = df_cleaned['parent_category_id'].astype(int)
df_cleaned['category_id'] = df_cleaned['category_id'].astype(int)
df_cleaned['orders'] = df_cleaned['orders'].apply(lambda x: order_clean(x))
df_cleaned['orders'] = df_cleaned['orders'].astype(int)
df_cleaned['package'] = df_cleaned['package'].fillna('-')
df_cleaned['brand'] = df_cleaned['brand'].fillna('-')
df_cleaned['product_name'] = df_cleaned['product_name'].apply(lambda x: x.lower())
df_cleaned['package'] = df_cleaned['package'].apply(lambda x: x.lower())
df_cleaned['brand'] = df_cleaned['brand'].apply(lambda x: x.lower())
df_cleaned['key'] = df_cleaned['product_name'] + "_" + df_cleaned['buy_unit'] + "_" + df_cleaned['package'] + "_" + df_cleaned['brand'] + "_" + df_cleaned['parent_category_id'].astype(str) + "_" + df_cleaned['category_id'].astype(str)

/var/folders/9k/2td9qcw563l80xbsy4yvr32c0000gn/T/ipykernel_43429/2549079233.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cleaned = df_len_ok.append(df_len_bad_10)


## Removing duplicated products using key

In [10]:
# Summing up orders of duplicated products 
df_aux = df_cleaned.groupby(['key']).agg({'orders':'sum'}).reset_index()

# Replacing orders to sum of orders
df_cleaned_dup = df_cleaned
df_cleaned_dup.drop_duplicates(subset ='key', keep = 'first', inplace = True)
df_cleaned_dup = df_cleaned_dup.drop(['orders'],axis=1)
df_cleaned_dup = pd.merge(df_cleaned_dup, df_aux ,how="left", on=['key'])
df_cleaned_dup = df_cleaned_dup.drop(['key'],axis=1)

In [11]:
print('''
No. of rows in the raw dataset: {}
No. of rows in the cleaned dataset: {}
--------------------------------------------
Data loss: {:.2f}%
'''.format(df_database.shape[0], df_cleaned_dup.shape[0], ((df_cleaned_dup.shape[0]/df_database.shape[0])-1)*100))


No. of rows in the raw dataset: 128081
No. of rows in the cleaned dataset: 106492
--------------------------------------------
Data loss: -16.86%



## Getting a list of duplicated products

In [12]:
df_list_dup = df_database
df_list_dup['key'] = df_list_dup['product_name'] + "_" + df_list_dup['buy_unit'] + "_" + df_list_dup['package'] + "_" + df_list_dup['brand'] + "_" + df_list_dup['parent_category_id'].astype(str) + "_" + df_list_dup['category_id'].astype(str)

df_list_dup =  df_database.groupby(['key', 'brand']).agg({'product_id':'count'}).reset_index()
df_list_dup = df_list_dup[(df_list_dup['product_id']) > 1]
df_list_dup.sort_values(by=['product_id'],ascending=False, inplace=True)
list_dup = df_list_dup['key'].tolist()
df_list_dup_final = df_database[df_database['key'].isin(list_dup)]
df_list_dup_final.sort_values(by=['key','orders'],ascending=False, inplace=True)
df_list_dup_final = df_list_dup_final.drop(['len','key'],axis=1)
df_list_dup_final['orders'] = df_list_dup_final['orders'].apply(lambda x: order_clean(x))
df_list_dup_final['product_id'] = df_list_dup_final['product_id'].astype(int)
df_list_dup_final['parent_category_id'] = df_list_dup_final['parent_category_id'].astype(int)
df_list_dup_final['category_id'] = df_list_dup_final['category_id'].astype(int)
df_list_dup_final

,product_id,product_name,buy_unit,package,brand,parent_category_id,category_id,parent_category_name,category_name,orders
65374,65372,Zuchinni org√°nico,UN,250 g,Hortifresco,538,136,Frutas y verduras,Verduras frescas,879
65378,65376,Zuchinni org√°nico,UN,250 g,Hortifresco,538,136,Frutas y verduras,Verduras frescas,279
48018,48016,Yogurt sligth,UN,200g,Colanta,512,24,L√°cteos y huevos,Yoghurt,94
48041,48039,Yogurt sligth,UN,200g,Colanta,512,24,L√°cteos y huevos,Yoghurt,907
48138,48136,Yogurt sligth,UN,200g,Colanta,512,24,L√°cteos y huevos,Yoghurt,608
...,...,...,...,...,...,...,...,...,...,...
90507,90505,Accesorios bisuteria,UN,2 un,Brilho,560,638,Librer√≠a,Bisuter√≠a y cuentas,528
90508,90506,Accesorios bisuteria,UN,2 un,Brilho,560,638,Librer√≠a,Bisuter√≠a y cuentas,513
90509,90507,Accesorios bisuteria,UN,2 un,Brilho,560,638,Librer√≠a,Bisuter√≠a y cuentas,448
29035,29033,Abrelatas b√°sico,UN,1 u,Imusa,551,1249,Ba√±os y cocina,Utensilios cocina,786


## Exporting file as xlsx

In [18]:
path_download = home + '/Downloads/DCA_test_Kokuda.xlsx'

with pd.ExcelWriter(path_download) as writer:
    df_cleaned_dup.to_excel(writer, sheet_name='dataset_cleaned',index=False)
    df_list_dup_final.to_excel(writer, sheet_name='list_of_duplications',index=False)

# This was used for discovery only

In [13]:
df_len_bad.groupby(['no_elements']).agg({'product_id':'count'}).reset_index()

,no_elements,product_id
0,1,15
1,2,17
2,4,12
3,5,1
4,7,10
5,9,16
6,10,1873
7,11,13861
8,12,3188
9,13,261


In [14]:
df_len_bad.head()

,product_id,product_name,buy_unit,package,brand,parent_category_id,category_id,parent_category_name,category_name,orders,len,split_list,no_elements
15,"16,At√∫n en aceite,UN,""6 un, 80 g c/u"",Van Cam...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,114,"[16, At√∫n en aceite, UN, ""6 un, 80 g c/u"", V...",11
31,"32,At√∫n vegetales,UN,""6 un, 150 g"",Van Camp's...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,111,"[32, At√∫n vegetales, UN, ""6 un, 150 g"", Van ...",11
33,"34,Atun aceite,UN,""12 un, 80 g"",Van Camp's,521...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,107,"[34, Atun aceite, UN, ""12 un, 80 g"", Van Camp...",11
88,"89,Cerveza,UN,""24 un, 269 mL"",Andina,515,40,Al...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,66,"[89, Cerveza, UN, ""24 un, 269 mL"", Andina, 51...",11
101,"102,Nestle Beso Negra 448g,UN,,Nestl√©,518,287...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,90,"[102, Nestle Beso Negra 448g, UN, , Nestl√©, 5...",11


In [15]:
x = df_len_bad[(df_len_bad['no_elements'] == 10)]
list_x = x['split_list'].tolist()[:5]
x = pd.DataFrame(list_x).T
x

,0,1,2,3,4
0,753,770,785,2412,3552
1,"""Silla De Rueda Drive Pedi√°trica 14""""""","""Silla de ruedas sport silver 18""""""","""Silla Ruedas Drive Transporte Acero 19""""""",Vaso de vidrio para licuadora \t4891-3,"""Ventilador personal negro 13"""""""
2,UN,UN,UN,UN,UN
3,,1 und,,1 und,2 u
4,Drive,Drive,Drive,Oster,Cascade
5,580,580,580,554,554
6,365,768,365,244,241
7,Salud,Salud,Salud,Electrohogar,Electrohogar
8,Equipos medici√≥n y dispositivos,Ortopedia,Equipos medici√≥n y dispositivos,Electrodom√©sticos,Ventilaci√≥n
9,198;;,493;;,368;;,208;;,720;;


In [16]:
x = df_len_bad[(df_len_bad['no_elements'] == 11)]
list_x = x['split_list'].tolist()[:5]
x = pd.DataFrame(list_x).T
x

,0,1,2,3,4
0,16,32,34,89,102
1,At√∫n en aceite,At√∫n vegetales,Atun aceite,Cerveza,Nestle Beso Negra 448g
2,UN,UN,UN,UN,UN
3,"""6 un","""6 un","""12 un","""24 un",
4,"80 g c/u""","150 g""","80 g""","269 mL""",Nestl√©
5,Van Camp's,Van Camp's,Van Camp's,Andina,518
6,521,521,521,515,287
7,83,83,83,40,"""Dulces"
8,Productos en conserva,Productos en conserva,Productos en conserva,Alcohol,"snacks y galletas"""
9,Carnes y mariscos en conserva,Carnes y mariscos en conserva,Carnes y mariscos en conserva,Cervezas,Chocolate


In [324]:
x = df_len_bad[(df_len_bad['no_elements'] == 12)]
list_x = x['split_list'].tolist()[:5]
x = pd.DataFrame(list_x).T
x

,0,1,2,3,4
0,131,585,610,612,649
1,Galleta rellena con crema de coco cocosette,Almendras rebanadas,"""Saz√≥n Completa Bad√≠a 49",Semillas de ch√≠a,Aloe cristalizada
2,UN,UN,"6 gr""",UN,UN
3,"""12 un",21.3 g,UN,42.5 g,28.3 g
4,"480 g""",Badia,"""49",Badia,Badia
5,Nestl√©,518,"6gr""",518,518
6,518,1004,Badia,1004,1004
7,981,"""Dulces",520,"""Dulces","""Dulces"
8,"""Dulces","snacks y galletas""",75,"snacks y galletas""","snacks y galletas"""
9,"snacks y galletas""","""Nueces",Despensa,"""Nueces","""Nueces"


In [343]:
test_len_ok = df_len_ok
test_len_ok['key1'] = test_len_ok['product_name'] + "_" + test_len_ok['buy_unit'] + "_" + test_len_ok['package'] + "_" + test_len_ok['brand'] + "_" + test_len_ok['parent_category_id'].astype(str) + "_" + test_len_ok['category_id'].astype(str)
test_len_ok['key2'] = test_len_ok['buy_unit'] + "_" + test_len_ok['package'] + "_" + test_len_ok['brand'] + "_" + test_len_ok['parent_category_id'].astype(str) + "_" + test_len_ok['category_id'].astype(str)
test_len_ok

,product_id,product_name,buy_unit,package,brand,parent_category_id,category_id,parent_category_name,category_name,orders,key1,key2
0,1,At√∫n en aceite,UN,3 u x 160 g c/u,Van Camp's,521.0,83.0,Productos en conserva,Carnes y mariscos en conserva,262;;,At√∫n en aceite_UN_3 u x 160 g c/u_Van Camp's_...,UN_3 u x 160 g c/u_Van Camp's_521.0_83.0
1,2,At√∫n en aceite,UN,1730 g,Van Camp's,521.0,83.0,Productos en conserva,Carnes y mariscos en conserva,387;;,At√∫n en aceite_UN_1730 g_Van Camp's_521.0_83.0,UN_1730 g_Van Camp's_521.0_83.0
2,3,At√∫n en aceite de girasol,UN,4 u x 160 g c/u,Van Camp's,521.0,83.0,Productos en conserva,Carnes y mariscos en conserva,600;;,At√∫n en aceite de girasol_UN_4 u x 160 g c/u_...,UN_4 u x 160 g c/u_Van Camp's_521.0_83.0
3,4,At√∫n en agua,UN,6 u x 160 g c/u,Van Camp's,521.0,83.0,Productos en conserva,Carnes y mariscos en conserva,749;;,At√∫n en agua_UN_6 u x 160 g c/u_Van Camp's_52...,UN_6 u x 160 g c/u_Van Camp's_521.0_83.0
4,5,At√∫n en agua,UN,3 u x 80 g c/u,Van Camp's,521.0,83.0,Productos en conserva,Carnes y mariscos en conserva,259;;,At√∫n en agua_UN_3 u x 80 g c/u_Van Camp's_521...,UN_3 u x 80 g c/u_Van Camp's_521.0_83.0
...,...,...,...,...,...,...,...,...,...,...,...,...
128076,128034,Cebolla larga,KG,Por peso,NaN,538,136,Frutas y verduras,Verduras frescas,720;;,NaN,NaN
128077,128035,Cilantro,UN,Paquete de 100 g,NaN,538,135,Frutas y verduras,Hierbas frescas,199;;,NaN,NaN
128078,128036,Frijol desgranado,KG,Por peso,NaN,538,136,Frutas y verduras,Verduras frescas,973;;,NaN,NaN
128079,128037,Guascas,UN,Paquete de 100 g,NaN,538,135,Frutas y verduras,Hierbas frescas,34;;,NaN,NaN


In [344]:
df_dsclean_dupcheck2 =  test_len_ok.groupby(['key2']).agg({'product_id':'count'}).reset_index()
df_dsclean_dupcheck2 = df_dsclean_dupcheck2[(df_dsclean_dupcheck2['product_id']) > 1]
df_dsclean_dupcheck2.sort_values(by=['product_id'],ascending=False, inplace=True)
df_dsclean_dupcheck2

,key2,product_id
35388,UN_Talla S_No-Varix_580_768,262
35413,UN_Talla XL_No-Varix_580_768,261
35345,UN_Talla M_No-Varix_580_768,258
35292,UN_Talla L_No-Varix_580_768,256
3877,UN_1.0_Bardot_527_974,185
...,...,...
15405,UN_3 und_Gum_527.0_967.0,2
15395,UN_3 und_Evergreen_1209.0_1228.0,2
15394,UN_3 und_Eterna_526.0_985.0,2
15392,UN_3 und_Durex_580.0_679.0,2


In [347]:
df_dsclean_dupcheck1 =  test_len_ok.groupby(['key1', 'brand']).agg({'product_id':'count'}).reset_index()
df_dsclean_dupcheck1 = df_dsclean_dupcheck1[(df_dsclean_dupcheck1['product_id']) > 1]
df_dsclean_dupcheck1.sort_values(by=['product_id'],ascending=False, inplace=True)
df_dsclean_dupcheck1

,key1,brand,product_id
61484,Vestido de fiesta mu√±ecas tipo american/our g...,Zassy,10
33487,Labial l√≠quido aca√≠ larga duraci√≥n_UN_6 ml_...,Vit√∫,9
61058,Vaso_UN_1 u_Aro_553_249,Aro,9
12330,Camiseta infantil dra clown_UN_Unidad_Dra Clow...,Dra Clown,8
33296,Labial color sensational_UN_4.2 g_Maybelline_5...,Maybelline,8
...,...,...,...
12003,Calcetin mujer transparente puntera abierta 15...,No-Varix,2
12002,Calcetin mujer transparente puntera abierta 15...,No-Varix,2
12001,Calcetin mujer transparente puntera abierta 15...,No-Varix,2
12000,Calcetin mujer transparente puntera abierta 15...,No-Varix,2
